# Writing an Algorithm that Uses DMAC Trading

## Data Loading

In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [2]:
# Read the aapl.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
aapl_df = pd.read_csv(
    Path("../Resources/aapl.csv"),
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True
)

In [3]:
# Review the DataFrame
aapl_df.head()

,close,volume,open,high,low
date,,,,,
2014-09-22,101.06,52421660,101.80,102.14,100.58
2014-09-23,102.64,63255860,100.60,102.94,100.54
2014-09-24,101.75,59974260,102.16,102.85,101.20
2014-09-25,97.87,99689300,100.51,100.71,97.72
2014-09-26,100.75,62276770,98.53,100.75,98.40


In [4]:
# Filter the date index and close columns
signals_df = aapl_df.loc[:,['close']]
# Review the DataFrame
signals_df

,close
date,
2014-09-22,101.06
2014-09-23,102.64
2014-09-24,101.75
2014-09-25,97.87
2014-09-26,100.75
...,...
2019-09-13,218.75
2019-09-16,219.90
2019-09-17,220.70


In [5]:
# Use hvplot to visualize the data
signals_df.hvplot()

:Curve   [date]   (close)

## SMA for the Short and Long Windows Calculation

In [6]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

In [7]:
# Generate the short and long window simple moving averages (50 and 100 days, respectively)
signals_df['sma50'] = signals_df.close.rolling(window=short_window).mean()
signals_df['sma100'] = signals_df.close.rolling(window=long_window).mean()
# Review the DataFrame
display(signals_df.head(100))
display(signals_df.head(100))

,close,sma50,sma100
date,,,
2014-09-22,101.06,NaN,NaN
2014-09-23,102.64,NaN,NaN
2014-09-24,101.75,NaN,NaN
2014-09-25,97.87,NaN,NaN
2014-09-26,100.75,NaN,NaN
...,...,...,...
2015-02-06,118.93,112.7422,NaN
2015-02-09,119.72,112.7846,NaN
2015-02-10,122.02,112.8450,NaN


,close,sma50,sma100
date,,,
2014-09-22,101.06,NaN,NaN
2014-09-23,102.64,NaN,NaN
2014-09-24,101.75,NaN,NaN
2014-09-25,97.87,NaN,NaN
2014-09-26,100.75,NaN,NaN
...,...,...,...
2015-02-06,118.93,112.7422,NaN
2015-02-09,119.72,112.7846,NaN
2015-02-10,122.02,112.8450,NaN


## Trading Signals Identification

In [8]:
# Create a column to hold the trading signal
signals_df['Signal'] = 0.0

In [9]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df['Signal'][short_window:] = np.where(signals_df["sma50"][short_window:] > signals_df['sma100'][short_window:],1, 0)
# Review the DataFrame
signals_df.head()

,close,sma50,sma100,Signal
date,,,,
2014-09-22,101.06,NaN,NaN,0.0
2014-09-23,102.64,NaN,NaN,0.0
2014-09-24,101.75,NaN,NaN,0.0
2014-09-25,97.87,NaN,NaN,0.0
2014-09-26,100.75,NaN,NaN,0.0


## Finding the Crossover Points

In [10]:
# Slice the DataFrame to confirm the Signal


In [11]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(50)

,close,sma50,sma100,Signal,Entry/Exit
date,,,,,
2019-07-11,201.750,193.7312,191.30140,1.0,0.0
2019-07-12,203.300,193.5868,191.62510,1.0,0.0
2019-07-15,205.210,193.5080,191.95690,1.0,0.0
2019-07-16,204.500,193.3630,192.29130,1.0,0.0
2019-07-17,203.350,193.2604,192.59510,1.0,0.0
2019-07-18,205.660,193.3164,192.90940,1.0,0.0
2019-07-19,202.590,193.3102,193.19200,1.0,0.0
2019-07-22,207.220,193.4402,193.51550,0.0,-1.0
2019-07-23,208.840,193.6734,193.87240,0.0,0.0


In [12]:
# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="yellow",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
exit

:Scatter   [date]   (close)

In [13]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
entry

:Scatter   [date]   (close)

In [14]:
# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
security_close

:Curve   [date]   (close)

In [16]:
# Visualize moving averages
moving_avgs = signals_df[["sma50", "sma100"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [17]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)